# View pcigale fit results


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2024-02-07
- uddate : 2024-02-07

Find pcigale here https://cigale.lam.fr



- adaptation : Sylvie Dagoret-Campagne from  https://cigale.lam.fr


In [ ]:
#%pylab widget
#import matplotlib.pyplot as plt
#matplotlib.rcParams['figure.figsize'] = [9.,6.]

In [ ]:
#%pylab widget
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.colors as colors
import matplotlib.cm as cmx
from matplotlib.colors import ListedColormap

import matplotlib.gridspec as gridspec

plt.rcParams['figure.figsize'] = [9.,6.]
import seaborn as sns

In [ ]:
#bwr_map = plt.get_cmap('bwr')
#reversed_map = bwr_map.reversed() 
#cNorm = colors.Normalize(0., vmax=1.)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=bwr_map)
#all_colors = scalarMap.to_rgba(df["zobs"].values, alpha=1)


In [ ]:
import numpy as np

from IPython.display import display
from ipywidgets import widgets
from matplotlib import pyplot as plt

from astropy.table import Table
from astropy.io import fits
from astropy import units as u
from astropy import constants as ctes 
from scipy.constants import c

from pcigale import sed
from pcigale import sed_modules as modules
from pcigale.warehouse import SedWarehouse
from pcigale.data import SimpleDatabase
import os,re

In [ ]:
c

In [ ]:
path = "20240205_181701_out"

In [ ]:
def get_fitresults_dict(the_path):
    """
    paramters:
     - path : path where pcigale write results
    """
    all_files_runpcigale = sorted(os.listdir(the_path))
    spec_dict = {}
    for idx,filename in enumerate(all_files_runpcigale):

        # skip all file that are not ended by fits
        if not re.search('.*[.]fits$',filename):
            continue

        # 1) decode the spec number
        str_spec_found = re.findall('(^SPEC.+?)_.*', filename)
        #print("\t", idx," ",filename," ", str_spec_found)
        if len( str_spec_found) == 0:
            print(f"Skip filename {filename}")
            continue
        str_spec_found = str_spec_found[0]
  

        if str_spec_found in spec_dict.keys():
            #print("Found in dict ", spec_dict[str_spec_found])
            pass
        else:
            #print(f" ! {str_spec_found} NOT found in dict ")
            spec_dict[str_spec_found] = {"best_model": None,"SFH":None}

        d = spec_dict[str_spec_found]
    
        # 2) decode thefilename
        str_file_found = re.findall('^SPEC.+_(.*)[.]fits$', filename)[0]
        #print(idx,filename,str_file_found)

        if str_file_found == "SFH":
            d["SFH"] = filename
        elif str_file_found == "model":
            d["best_model"] = filename

        spec_dict[str_spec_found] = d
        
    return spec_dict
               
    

## Input

### Dictionary of file

In [ ]:
spec_dict = get_fitresults_dict(path)

### Select spectrum

In [ ]:
index = 10
spec_name = list(spec_dict.keys())[index]
spec_name

In [ ]:
spec_dict[spec_name]

In [ ]:
file_best_model = os.path.join(path,spec_dict[spec_name]['best_model'])
file_sfh = os.path.join(path,spec_dict[spec_name]['SFH'])

In [ ]:
t_best = Table.read(file_best_model)
t_sfh = Table.read(file_sfh)

In [ ]:
t_best[:100]

In [ ]:
list(t_best.columns)

## Read all results

In [ ]:
t_res = Table.read(os.path.join(path,'results.fits'))

In [ ]:
t_res[:4]

### Selection 

In [ ]:
cut_select = t_res["id"] == spec_name

In [ ]:
t_res_sel = t_res[cut_select]
t_res_sel

In [ ]:
t_res_sel["best.reduced_chi_square"][0]

### Read data

In [ ]:
file_data = os.path.join(path,"observations.fits")
FORS2= Table.read(file_data)
FORS2.add_index('id')

In [ ]:
FORS2[:4]

### Select

In [ ]:
cut_fors2select = FORS2["id"] == spec_name

In [ ]:
FORS2_sel = FORS2[cut_fors2select]

In [ ]:
FORS2_sel["galex.FUV"]

### Filters

In [ ]:
BANDS = [
    col for col in FORS2.colnames if not col.endswith("_err") 
    and not col.startswith("line") 
    and not col in ["id", "redshift", "distance"]
]

#with Database() as d:
#    BANDS_WAVE = np.array([d.get_filter(name).pivot_wavelength for name in BANDS])


with SimpleDatabase('filters') as d:
    #BANDS_WAVE = np.array([d.get(name).pivot_wavelength for name in BANDS])
    BANDS_WAVE = np.array([d.get(name=name).pivot for name in BANDS])
    

In [ ]:
BANDS_best = [ "best." + bandname for bandname in BANDS]

In [ ]:
band_sed_fluxes = np.array([ t_res_sel[bandname][0] for bandname in  BANDS_best])

In [ ]:
#list_of_columns = list(t_res_sel.columns)
#for idx,col in enumerate(list_of_columns):
#    print(idx,col)

In [ ]:
# Initial values of the SED parameters
SED_PARAMETERS = {
    'sfhdelayed': {
        'tau_main': t_res_sel["best.sfh.tau_main"][0],
        'age_main': t_res_sel["best.sfh.age_main"][0],
        'tau_burst': t_res_sel["best.sfh.tau_burst"][0],
        'age_burst': t_res_sel["best.sfh.age_burst"][0],
        'f_burst': t_res_sel["best.sfh.f_burst"][0],
        },
    'bc03': {
        'imf': t_res_sel["best.stellar.imf"][0],
        'metallicity': t_res_sel["best.stellar.metallicity"][0],
    },
    'nebular': {
        'logU': t_res_sel["best.nebular.logU"][0],
        'f_esc': t_res_sel["best.nebular.f_esc"][0],
        'f_dust': t_res_sel["best.nebular.f_dust"][0],
        'emission': True,
    },
    'dustatt_modified_starburst': {
        'E_BV_lines': t_res_sel["best.attenuation.E_BV_lines"][0],
        'E_BV_factor': t_res_sel["best.attenuation.E_BV_factor"][0],
        'uv_bump_wavelength': t_res_sel["best.attenuation.uv_bump_wavelength"][0],
        'uv_bump_width': t_res_sel["best.attenuation.uv_bump_width"][0],
        'uv_bump_amplitude': t_res_sel["best.attenuation.uv_bump_amplitude"][0],
        'powerlaw_slope': t_res_sel["best.attenuation.powerlaw_slope"][0],
        'Ext_law_emission_lines': 1,
        'Rv': 3.1,
    },
    'dl2014': {
        'qpah': 2.50,
        'umin': 1.5,
        'alpha': 2.0,
        'gamma': 0.02,
    },
    'redshifting': {
        'redshift': t_res_sel["best.universe.redshift"][0],
    }
}

In [ ]:
t_best[:3]

In [ ]:
sed = t_best
print(sed["stellar.young"].data)

In [ ]:
cmap = ListedColormap(sns.color_palette("hls",10))   

In [ ]:
cmap.colors

In [ ]:
cmap.colors[0]

In [ ]:
def plot_allSEDS(spec_name,t_res_sel,t_best,fors2_sel,band_sed_fluxes,BANDS,BANDS_WAVE,x_lims = [100.,5e4],y_lims=[1e31,1e35]):
    
    fig,(ax1,ax2) = plt.subplots(2,1,figsize=(12, 10),sharex=True)
    
   
    # figure 1
    
    # decode the result of the fit
    chi2_red = t_res_sel["best.reduced_chi_square"][0]
    redshift = t_res_sel["best.universe.redshift"][0]	


    sed_wavelength_grid = t_best["wavelength"].data
    sed_fnu = t_best["Fnu"].data

    # the SED
    ax1.grid()
    ax1.loglog(sed_wavelength_grid, sed_fnu  ,'b-' ,label="Fitted SED")

    # the model
    ax1.scatter(BANDS_WAVE, band_sed_fluxes, color="r",label="Fitted SED band fluxes")

    # the data
    band_gal_fluxes = np.array([FORS2.loc[spec_name][band] for band in BANDS])
    band_gal_err = [FORS2.loc[spec_name][f"{band}_err"] for band in BANDS]
    ax1.errorbar(BANDS_WAVE, band_gal_fluxes, yerr=band_gal_err, fmt='o',color="k" ,label=f"{spec_name} fluxes, chi2_red = {chi2_red:.3f}")


    mask_sed = (sed_wavelength_grid >= x_lims[0]) & (sed_wavelength_grid <= x_lims[1])
    mask_bands = (BANDS_WAVE >= x_lims[0]) & (BANDS_WAVE <= x_lims[1])


    y1 = np.min(sed_fnu[mask_sed])
    y2 = np.min(band_sed_fluxes[mask_bands])

    # handle flux not Nan
    data_fluxes = band_gal_fluxes[mask_bands]
    data_fluxes = data_fluxes[~np.isnan(data_fluxes)]
    y3 = np.min(data_fluxes)

    y4 = np.max(sed_fnu[mask_sed])
    y5 = np.max(band_sed_fluxes[mask_bands])
    y6 = np.max(data_fluxes)

  
    y_min = 0.8 * np.min([y1,y2,y3])
    if y_min <= 0:
        y_min = 1e-30  # log axis
    y_max = 1.1 * np.max([y4,y5,y6])

    ax1.set_ylim(y_min,y_max)
    ax1.set_xlim(x_lims[0],x_lims[1])

    ax1.legend(loc=0)
    ax1.set_ylabel ("Flux [mJy]")
    ax1.set_xlabel("Wavelength [nm]") 
    ax1.set_title(f"Fors2 Flux {spec_name} , z = {redshift:.3f}, in pcigale run {path}")



    list_of_columns = list(t_best.columns)
    ncols = len(list_of_columns)

    # convert seaborn palette into a matplotlib palette
    cmap = ListedColormap(sns.color_palette("hls",ncols))   
   
    all_ymin = []
    all_ymax = []
    
    for idx in range(2,ncols):
        colname = list_of_columns[idx]
        ax2.loglog(sed_wavelength_grid[mask_sed],t_best[colname].data[mask_sed],"-",color=cmap.colors[idx-1],label=colname)
        ymin = np.min(t_best[colname].data[mask_sed])
        ymax = np.min(t_best[colname].data[mask_sed])

        if not np.isnan(ymin) and ymin>0 :
            all_ymin.append(ymin)
        if not np.isnan(ymax) and ymax>0:    
            all_ymax.append(ymax)

   
    ymax = np.min(all_ymax)*1.2
    ymin = np.min(all_ymin)*0.8
    ymin = np.max([ymin,ymax/1e6])
    #ymin = ymax/1e6
    #print(ymin,ymax)

    ax2.grid()
    ax2.set_xlim(x_lims[0],x_lims[1])
    ax2.set_ylim(y_lims[0],y_lims[1])

    ax2.legend(loc=0)
    ax2.set_ylabel ("Luminosity [W/nm]")
    ax2.set_xlabel("Wavelength [nm]") 

    
    plt.tight_layout()
    _ = plt.show()

In [ ]:
plot_allSEDS(spec_name,t_res_sel,t_best,FORS2_sel,band_sed_fluxes,BANDS,BANDS_WAVE)

# From pcigale

In [ ]:
with SimpleDatabase('filters') as d:
    #BANDS_WAVE = np.array([d.get(name).pivot_wavelength for name in BANDS])
    BANDS_WAVE = np.array([d.get(name=name).pivot for name in BANDS])
    BANDS_NAME = BANDS
dict_filters = dict(zip(BANDS_NAME, BANDS_WAVE)) 

In [ ]:
dict_filters 

In [ ]:
#obs= FORS2_sel.to_pandas().to_dict()
obs = FORS2_sel
obs

In [ ]:
 t_res_sel["best.universe.luminosity_distance"].data[0]

In [ ]:
t_best.colnames

In [ ]:
series = ["model" ,"stellar_attenuated","stellar_unattenuated","nebular","dust"]

In [ ]:
sed = t_best
sed.columns[0].data
sed["stellar.young"].data

In [ ]:
c/ctes.c

In [ ]:
ctes.c

In [ ]:
def plots_frompcigale(t_best_model, mod,obs, filters , series=series,sed_type = "mJy", xrange = [100.*1e-3,1e4*1e-3],yrange = [False,False]):
    """

    parameters :
       xrange : wavelength ranage in microns xrange = [100.*1e-3,1e4*1e-3]
    
    Need to convert flux density in mJy (mW/m2/Hz) into Luminosity W/nm and vice versa

    Fnu = 1/(4piDL^2) * lambda^2/c * L_lambda
    L_lambda = (4piDL^2) * (c/lambda^2) * F_nu

    No K correction is calculated
    
    """

    np.seterr(invalid="ignore")

    if True:
        
        sed = t_best_model
        # wavelengths from nm to microns
        filters_wl = (np.array([filt_wl for filt_wl in filters.values()]) * 1e-3)

        # the best model
        # wavelengths from nm to microns
        
        wavelength_spec = sed["wavelength"].data * 1e-3
      
        obs_fluxes = np.array([obs[filt].data[0] for filt in filters.keys()])
        obs_fluxes_err = np.array([obs[filt + "_err"].data[0] for filt in filters.keys()])
        mod_fluxes = np.array(
            [
                mod["best." + filt].data[0]
                if "best." + filt in mod.colnames
                else np.nan
                for filt in filters.keys()
            ])

        
        
        if obs["redshift"].data[0] >= 0:
            z = float(obs["redshift"].data[0])
        else:  # Redshift mode
            z = float(mod["best.universe.redshift"].data[0])

    
        
        zp1 = 1.0 + z

        # surface of the sphere centered on the object of radius distance-luminosity
        surf = 4.0 * np.pi * mod["best.universe.luminosity_distance"].data[0] ** 2

        

        xmin = 0.9 * np.min(filters_wl) if xrange[0] is False else xrange[0]
        xmax = 1.1 * np.max(filters_wl) if xrange[1] is False else xrange[1]

        if sed_type == "lum":
            #L_lambda = (4piDL^2) * (c/lambda^2) * F_nu
            # 1 mJy = 10^-29 W/m2/Hz
            # Every values converted to SI 
            # wavelength from microns to meters
            # need to convert lumi from W/m to W/nm
            k_corr_SED = (1e-29 * surf * ctes.c.value/ (filters_wl * 1e-6)**2)*1e-9

            print("k_corr_SED --> W/m2 = ",k_corr_SED)

            print(z,surf,k_corr_SED)
            # convert fluxes from mJy to W/nm
            obs_fluxes *= k_corr_SED
            obs_fluxes_err *= k_corr_SED
            mod_fluxes *= k_corr_SED

            #for cname in sed.colnames[1:]:
            for cname in sed.colnames[2:]:
                sed[cname] *= wavelength_spec * 1e3
            # set source rest frame
            filters_wl /= zp1
            wavelength_spec /= zp1
            xmin /= zp1
            xmax /= zp1
        elif sed_type == "mJy":
            k_corr_SED = 1.0
            # 1 W/m2/Hz = 10^29 mJy
            #Fnu = 1/(4piDL^2) * lambda^2/c * L_lambda
            fact = 1e29 * 1e-3 * wavelength_spec ** 2 / ctes.c.value / surf

            print("fact --> mJy = ",fact)
            
            #for cname in sed.colnames[1:]:
            for cname in sed.colnames[2:]:
                sed[cname] *= fact
        else:
            console.print(f"{ERROR} Unknown plot type.")

        wsed = np.where((wavelength_spec > xmin) & (wavelength_spec < xmax))
       

        print("wavelengths = ",wavelength_spec[wsed] )
        # START the figure
        
        print("================ START the figure ================================================================")

        #series = ["model" ,"stellar_attenuated","stellar_unattenuated","nebular","dust"]

        #'wavelength',
        # 'Fnu',
        # 'L_lambda_total',
        # 'stellar.old',
        # 'stellar.young',
        # 'nebular.absorption_old',
        # 'nebular.absorption_young',
        # 'nebular.lines_old',
        # 'nebular.lines_young',
        # 'nebular.continuum_old',
        # 'nebular.continuum_young',
        # 'attenuation.stellar.old',
        # 'attenuation.stellar.young',
        # 'attenuation.nebular.lines_old',
        # 'attenuation.nebular.lines_young',
        # 'attenuation.nebular.continuum_old',
        # 'attenuation.nebular.continuum_young',
        # 'dust',
        # 'igm'


        
        figure = plt.figure()
        gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
        
        #if (sed.columns[1][wsed] > 0.0).any():
        if (sed.columns[0][wsed].data > 0.0).any():    
            
            ax1 = plt.subplot(gs[0])
            ax2 = plt.subplot(gs[1])

            # Stellar emission
            if ( "stellar_attenuated" in series and "stellar.young" in sed.columns):
                spectrum =  sed["stellar.young"].data[wsed]  + sed["stellar.old"].data[wsed]

                if "nebular.absorption_young" in sed.columns:
                    spectrum += sed["nebular.absorption_young"].data[wsed]
                    spectrum += sed["nebular.absorption_old"].data[wsed]

                if "attenuation.stellar.young" in sed.columns:
                    spectrum += sed["attenuation.stellar.young"].data[wsed]
                    spectrum += sed["attenuation.stellar.old"].data[wsed]

                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Stellar attenuated",
                        color="gold",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )
                print("stellar_attenuated spectrum \t",spectrum)

            if ("stellar_unattenuated" in series and "stellar.young" in sed.columns):

                spectrum = sed["stellar.old"].data[wsed] + sed["stellar.young"].data[wsed]
                
                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Stellar unattenuated",
                        color="xkcd:deep sky blue",
                        marker=None,
                        nonpositive="clip",
                        linestyle="--",
                        linewidth=1.0,
                    )
                print("stellar_unattenuated spectrum \t",spectrum)

            # Nebular emission
            if "nebular" in series and "nebular.lines_young" in sed.columns:
                spectrum = (
                        sed["nebular.lines_young"].data[wsed]
                        + sed["nebular.lines_old"].data[wsed]
                        + sed["nebular.continuum_young"].data[wsed]
                        + sed["nebular.continuum_old"].data[wsed]
                    )

                if "attenuation.nebular.lines_young" in sed.columns:
                        spectrum += sed["attenuation.nebular.lines_young"].data[wsed]
                        spectrum += sed["attenuation.nebular.lines_old"].data[wsed]
                        spectrum += sed["attenuation.nebular.continuum_young"].data[wsed]
                        spectrum += sed["attenuation.nebular.continuum_old"].data[wsed]

                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Nebular emission",
                        color="xkcd:true green",
                        marker=None,
                        nonpositive="clip",
                        linewidth=1.0,
                    )
                print("nebular  spectrum \t",spectrum)

            # Dust emission Draine & Li
            if "dust" in series and "dust.Umin_Umin" in sed.columns:
                spectrum =  sed["dust.Umin_Umin"].data[wsed] + sed["dust.Umin_Umax"].data[wsed]
                ax1.loglog(
                        wavelength_spec[wsed],spectrum,
                        label="Dust emission",
                        color="xkcd:bright red",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )
                print("Dust Draine Li  spectrum \t",spectrum)

            # Dust emission Dale
            if "dust" in series and "dust" in sed.columns:
                spectrum = sed["dust"].data[wsed]
                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Dust emission",
                        color="xkcd:bright red",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )
                print("Dust spectrum \t",spectrum)

            # AGN emission
            if "agn" in series and ( "agn.fritz2006_torus" in sed.columns or "agn.SKIRTOR2016_torus" in sed.columns):
                if "agn.fritz2006_torus" in sed.columns:
                    agn_sed = (
                            sed["agn.fritz2006_polar_dust"]
                            + sed["agn.fritz2006_torus"]
                            + sed["agn.fritz2006_disk"]
                        )
                elif "agn.SKIRTOR2016_torus" in sed.columns:
                    agn_sed = (
                            sed["agn.SKIRTOR2016_polar_dust"]
                            + sed["agn.SKIRTOR2016_torus"]
                            + sed["agn.SKIRTOR2016_disk"]
                        )
                if "xray.agn" in sed.columns:
                    agn_sed += sed["xray.agn"]
                if "radio.agn" in sed.columns:
                    agn_sed += sed["radio.agn"]
                    
                ax1.loglog(
                        wavelength_spec[wsed],
                        agn_sed[wsed],
                        label="AGN emission",
                        color="xkcd:apricot",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )

            # Radio emission
            if "radio" in series and "radio.sf_nonthermal" in sed.columns:
                ax1.loglog(
                        wavelength_spec[wsed],
                        sed["radio.sf_nonthermal"][wsed],
                        label="Radio SF nonthermal",
                        color="brown",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )

            if "model" in series:
                ax1.loglog(
                        wavelength_spec[wsed],
                        sed["L_lambda_total"].data[wsed],
                        label="Model spectrum",
                        color="k",
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.5,
                    )
                print("model", sed["L_lambda_total"].data[wsed])

            
            ax1.set_autoscale_on(False)
            
            s = np.argsort(filters_wl)
            filters_wl = filters_wl[s]
            mod_fluxes = mod_fluxes[s]
            obs_fluxes = obs_fluxes[s]
            obs_fluxes_err = obs_fluxes_err[s]
            ax1.scatter(
                    filters_wl,
                    mod_fluxes,
                    marker="o",
                    color="xkcd:strawberry",
                    s=8,
                    zorder=3,
                    label="Model fluxes",
                )
            mask_ok = np.logical_and(obs_fluxes > 0.0, obs_fluxes_err > 0.0)
            ax1.errorbar(
                    filters_wl[mask_ok],
                    obs_fluxes[mask_ok],
                    yerr=obs_fluxes_err[mask_ok],
                    ls="",
                    marker="o",
                    label="Observed fluxes",
                    markerfacecolor="None",
                    markersize=5,
                    markeredgecolor="xkcd:pastel purple",
                    color="xkcd:light indigo",
                    capsize=0.0,
                    zorder=3,
                    lw=1,
                )
            mask_uplim = np.logical_and(
                    np.logical_and(obs_fluxes > 0.0, obs_fluxes_err < 0.0),
                    obs_fluxes_err > -9990.0 * k_corr_SED,
                )
            if not mask_uplim.any() == False:
                ax1.errorbar(
                        filters_wl[mask_uplim],
                        obs_fluxes[mask_uplim],
                        yerr=obs_fluxes_err[mask_uplim],
                        ls="",
                        marker="v",
                        label="Observed upper limits",
                        markerfacecolor="None",
                        markersize=6,
                        markeredgecolor="g",
                        capsize=0.0,
                    )
            mask_noerr = np.logical_and( obs_fluxes > 0.0, obs_fluxes_err < -9990.0 * k_corr_SED)
            if not mask_noerr.any() == False:
                ax1.errorbar(
                        filters_wl[mask_noerr],
                        obs_fluxes[mask_noerr],
                        ls="",
                        marker="p",
                        markerfacecolor="None",
                        markersize=5,
                        markeredgecolor="r",
                        label="Observed fluxes, no errors",
                        capsize=0.0,
                    )
            mask = np.where(obs_fluxes > 0.0)
            ax2.errorbar(
                    filters_wl[mask],
                    (obs_fluxes[mask] - mod_fluxes[mask]) / obs_fluxes[mask],
                    yerr=obs_fluxes_err[mask] / obs_fluxes[mask],
                    marker="_",
                    label="(Obs-Mod)/Obs",
                    color="k",
                    capsize=0.0,
                    ls="None",
                    lw=1,
                )
            ax2.plot([xmin, xmax], [0.0, 0.0], ls="--", color="k")
            ax2.set_xscale("log")
            ax2.minorticks_on()

            ax1.tick_params(
                    direction="in",
                    axis="both",
                    which="both",
                    top=True,
                    left=True,
                    right=True,
                    bottom=False,
                )
            ax2.tick_params(direction="in", axis="both", which="both", right=True)

            figure.subplots_adjust(hspace=0.0, wspace=0.0)

            ax1.set_xlim(xmin, xmax)

            if yrange[0] is not False:
                ymin = yrange[0]
            else:
                if not mask_uplim.any() == False:
                    ymin = min(
                            min(
                                np.min(obs_fluxes[mask_ok]),
                                np.min(obs_fluxes[mask_uplim]),
                            ),
                            min(
                                np.min(mod_fluxes[mask_ok]),
                                np.min(mod_fluxes[mask_uplim]),
                            ),
                        )
                elif not mask_ok.any() == False:
                    ymin = min(
                            np.min(obs_fluxes[mask_ok]),
                            np.min(mod_fluxes[mask_ok]),
                        )
                else:  # No valid flux (e.g., fitting only properties)
                    ymin = ax1.get_ylim()[0]
                ymin *= 1e-1

            if yrange[1] is not False:
                ymax = yrange[1]
            else:
                if not mask_uplim.any() == False:
                    ymax = max(
                            max(
                                np.max(obs_fluxes[mask_ok]),
                                np.max(obs_fluxes[mask_uplim]),
                            ),
                            max(
                                np.max(mod_fluxes[mask_ok]),
                                np.max(mod_fluxes[mask_uplim]),
                            ),
                        )
                elif not mask_ok.any() == False:
                    ymax = max(
                            np.max(obs_fluxes[mask_ok]),
                            np.max(mod_fluxes[mask_ok]),
                        )
                else:  # No valid flux (e.g., fitting only properties)
                    ymax = ax1.get_ylim()[1]
                ymax *= 1e5

            xmin = xmin if xmin < xmax else xmax - 1e1
            ymin = ymin if ymin < ymax else ymax - 1e1

            ax1.set_ylim(ymin, ymax)
            ax2.set_xlim(xmin, xmax)
            ax2.set_ylim(-1.0, 1.0)
            if sed_type == "lum":
                ax2.set_xlabel(r"Rest-frame wavelength [$\mu$m]")
                ax1.set_ylabel("Luminosity [W/nm]")
            else:
                ax2.set_xlabel(r"Observed $\lambda$ ($\mu$m)")
                ax1.set_ylabel(r"S$_\nu$ (mJy)")
            ax2.set_ylabel("Relative\nresidual")
            ax1.legend(fontsize=6, loc="best", frameon=False)
            ax2.legend(fontsize=6, loc="best", frameon=False)
            plt.setp(ax1.get_xticklabels(), visible=False)
            plt.setp(ax1.get_yticklabels()[1], visible=False)
            figure.suptitle(
                    f"Best model for {obs['id'].data[0]}\n (z={z:.3}, "
                    f"reduced χ²={mod['best.reduced_chi_square'].data[0]:.2})"
                )


        plt.show()
        #plt.close(figure)
      


In [ ]:
plots_frompcigale(t_best_model= t_best, mod = t_res_sel, obs= FORS2_sel, filters = dict_filters, series=series, 
                  sed_type = "mJy")

In [ ]:
assert False

In [ ]:
plt.figure(figsize=(12, 7))

x_lims = [100.,5e4]
chi2_red = t_res_sel["best.reduced_chi_square"][0]
redshift = t_res_sel["best.universe.redshift"][0]	


sed_wavelength_grid = t_best["wavelength"].data
sed_fnu = t_best["Fnu"].data


plt.grid()
plt.loglog(sed_wavelength_grid, sed_fnu  ,'b-' ,label="Fitted SED")

# the model
plt.scatter(BANDS_WAVE, band_sed_fluxes, color="r",label="Fitted SED band fluxes")

# the data
band_gal_fluxes = np.array([FORS2.loc[spec_name][band] for band in BANDS])
band_gal_err = [FORS2.loc[spec_name][f"{band}_err"] for band in BANDS]
plt.errorbar(BANDS_WAVE, band_gal_fluxes, yerr=band_gal_err, fmt='o',color="k" ,label=f"{spec_name} fluxes, chi2_red = {chi2_red:.3f}")


mask_sed = (sed_wavelength_grid >= x_lims[0]) & (sed_wavelength_grid <= x_lims[1])
mask_bands = (BANDS_WAVE >= x_lims[0]) & (BANDS_WAVE <= x_lims[1])


y1 = np.min(sed_fnu[mask_sed])
y2 = np.min(band_sed_fluxes[mask_bands])

# handle flux not Nan
data_fluxes = band_gal_fluxes[mask_bands]
data_fluxes = data_fluxes[~np.isnan(data_fluxes)]
y3 = np.min(data_fluxes)

y4 = np.max(sed_fnu[mask_sed])
y5 = np.max(band_sed_fluxes[mask_bands])
y6 = np.max(data_fluxes)

  
y_min = 0.8 * np.min([y1,y2,y3])
if y_min <= 0:
    y_min = 1e-30  # log axis
y_max = 1.1 * np.max([y4,y5,y6])

plt.ylim(y_min,y_max)
plt.xlim(x_lims[0],x_lims[1])

plt.legend(loc=0)
plt.ylabel ("Flux [Jy]")
plt.xlabel("Wavelength [nm]")
plt.tight_layout()
plt.title(f"Fors2 Spectrum {spec_name} , z = {redshift:.3f}, in pcigale run {path}")
_ = plt.show()

In [ ]:
ctes.c.value

In [ ]:
u.Jansky.decompose()/(ctes.c)**2

In [ ]:
1e-3 * u.Jansky.to(u.W/(u.m)**2, equivalencies=u.spectral_densities(3500 * u.AA))

In [ ]:
WAREHOUSE = SedWarehouse()

def plot_sed(name, log_adjust_factor, tau_main, age_main, tau_burst, age_burst, f_burst, metallicity, logU,
             f_esc, f_dust, E_BV_lines, E_BV_factor, uv_bump_width, uv_bump_amplitude, powerlaw_slope, Rv, qpah, 
             umin, alpha, gamma, redshift, l_range):

    chi2_red = t_res_sel["best.reduced_chi_square"][0]
    redshift = t_res_sel["best.universe.redshift"][0]	
    
    plt.figure("Interactive_SED", figsize=(9, 7))
    sed = WAREHOUSE.get_sed(
        module_list = ['sfhdelayed', 'bc03', 'nebular', 'dustatt_modified_starburst', 'dl2014', 'redshifting'],
        parameter_list= [
            {
                'tau_main': tau_main,
                'age_main': age_main,
                'tau_burst': tau_burst,
                'age_burst': age_burst,
                'f_burst': f_burst,
            },{
                'imf': 0,
                'metallicity': metallicity,
            },{
                'logU': logU,
                'f_esc': f_esc,
                'f_dust': f_dust,
                'emission': True,
            },{
                'E_BV_lines': E_BV_lines,
                'E_BV_factor': E_BV_factor,
                'uv_bump_wavelength': 217.5,
                'uv_bump_width': uv_bump_width,
                'uv_bump_amplitude': uv_bump_amplitude,
                'powerlaw_slope': powerlaw_slope,
                'Ext_law_emission_lines': 1,
                'Rv': Rv,
            },{
                'qpah': qpah,
                'umin': umin,
                'alpha': alpha,
                'gamma': gamma,
            }, {
                'redshift': redshift,
            }
        ]
    )
    
    adjust_factor = 10**log_adjust_factor

    # the model
    band_sed_fluxes = np.array([adjust_factor * sed.compute_fnu(band) for band in BANDS])

    # the data
    band_gal_fluxes = np.array([FORS2.loc[name][band] for band in BANDS])
    band_gal_err = [FORS2.loc[name][f"{band}_err"] for band in BANDS]
    
    x_lims = (10**l_range[0], 10**l_range[1])
    plt.clf()
    plt.grid()
    plt.loglog(sed.wavelength_grid, adjust_factor * sed.fnu ,'b-' ,label="SED")
    plt.scatter(BANDS_WAVE, band_sed_fluxes, color="r",label="SED band fluxes")
    plt.errorbar(BANDS_WAVE, band_gal_fluxes, yerr=band_gal_err, fmt='o',color="k" ,label=f"{name} fluxes, chi2_red = {chi2_red:.3f}")
    plt.xlim(x_lims)
    # Recompute the y limits
    
    mask_sed = (sed.wavelength_grid >= x_lims[0]) & (sed.wavelength_grid <= x_lims[1])
    mask_bands = (BANDS_WAVE >= x_lims[0]) & (BANDS_WAVE <= x_lims[1])

    y1 = np.min(adjust_factor * sed.fnu[mask_sed])
    y2 = np.min(band_sed_fluxes[mask_bands])

    # handle flux not Nan
    data_fluxes = band_gal_fluxes[mask_bands]
    data_fluxes = data_fluxes[~np.isnan(data_fluxes)]
    y3 = np.min(data_fluxes)

    y4 = np.max(adjust_factor * sed.fnu[mask_sed])
    y5 = np.max(band_sed_fluxes[mask_bands])
    y6 = np.max(data_fluxes)

  
    y_min = 0.9 * np.min([y1,y2,y3])
    if y_min <= 0:
        y_min = 1e-40  # log axis
    y_max = 1.1 * np.max([y4,y5,y6])

    
    plt.ylim((y_min, y_max))
    plt.legend(loc=0)
    plt.ylabel ("Flux [Jy]")
    plt.xlabel("Wavelength [nm]")
    plt.tight_layout()
    plt.title(f"Fors2 Spectrum {name} , z = {redshift:.3f}, in pcigale run {path}")
    _ = plt.show()

In [ ]:
plt.close("Interactive_SED")  # needed to rerun the cell

name = widgets.Dropdown(value=FORS2_sel['id'][0], 
                        options=list(FORS2_sel['id']))

log_adjust_factor = widgets.FloatSlider(11, min=0, max=16, step=0.1)

tau_main = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['tau_main'], 
                               min=500, max=8000, step=100, 
                               description="tau_main", continuous_update=False)
age_main = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['age_main'], 
                               min=2000, max=10000, step=1000, 
                               description="age_main", continuous_update=False)
tau_burst = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['tau_burst'], 
                                min=500, max=40000, step=100, 
                                description="tau_burst", continuous_update=False)
age_burst = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['age_burst'],
                                min=50, max=500, step=100, 
                                description="age_burst", continuous_update=False)
f_burst = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['f_burst'], 
                              min=0, max=.9, step=.1, 
                              description="f_burst", continuous_update=False)

metallicity = widgets.Dropdown(value=SED_PARAMETERS['bc03']['metallicity'], 
                               options=[0.0001, 0.0004, 0.004, 0.008, 0.02, 0.05], 
                               description="metallicity")

logU = widgets.Dropdown(value=SED_PARAMETERS['nebular']['logU'],
                        options=[-4.0, -3.9, -3.8, -3.7, -3.6, -3.5, -3.4, -3.3, 
                                 -3.2, -3.1, -3.0, -2.9, -2.8, -2.7, -2.6, -2.5, 
                                 -2.4, -2.3, -2.2, -2.1, -2.0, -1.9, -1.8, -1.7,
                                 -1.6, -1.5, -1.4, -1.3, -1.2, -1.1, -1.0],
                        description="logU")
f_esc = widgets.FloatSlider(value=SED_PARAMETERS['nebular']['f_esc'], 
                            min=0, max=1, step=.1, 
                            description="f_esc", continuous_update=False)
f_dust = widgets.FloatSlider(value=SED_PARAMETERS['nebular']['f_dust'], 
                             min=0, max=1, step=.1, 
                             description="f_dust", continuous_update=False)
# f_esc + f_dust is at most 1 (the remaining part is emitted as lines)
def update_f_esc(*args):
    if f_esc.value + f_dust.value > 1:
        f_esc.value = 1 - f_dust.value
f_dust.observe(update_f_esc, 'value')
def update_f_dust(*args):
    if f_esc.value + f_dust.value > 1:
        f_dust.value = 1 - f_esc.value
f_esc.observe(update_f_dust, 'value')

E_BV_lines = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['E_BV_lines'],
                                 min=0, max=1, step=.05,
                                 description='E_BV_lines', continuous_update=False)
E_BV_factor = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['E_BV_factor'],
                                 min=0, max=1, step=.1,
                                 description='E_BV_factor', continuous_update=False)
uv_bump_amplitude = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['uv_bump_amplitude'], 
                                        min=0, max=5, step=0.1, 
                                        description="bump_ampl.", continuous_update=False)
uv_bump_width = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['uv_bump_width'],
                                    min=100, max=500, step=100, 
                                    description="bump_width", continuous_update=False)

powerlaw_slope  = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['powerlaw_slope'],
                                      min=-1, max=1, step=.1, 
                                      description="slope", continuous_update=False)
Rv = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['Rv'],
                         min=0, max=5, step=.1, 
                         description="Rv", continuous_update=False)

qpah = widgets.Dropdown(value=SED_PARAMETERS['dl2014']['qpah'],
                        options=[0.47, 1.12, 1.77, 2.50, 3.19, 3.90, 4.58, 5.26, 5.95, 
                                 6.63, 7.32],
                        description="qpah")
umin = widgets.Dropdown(value=SED_PARAMETERS['dl2014']['umin'],
                        options=[0.100, 0.120, 0.150, 0.170, 0.200, 0.250, 0.300, 0.350, 
                                 0.400, 0.500, 0.600, 0.700, 0.800, 1.000, 1.200, 1.500, 
                                 1.700, 2.000, 2.500, 3.000, 3.500, 4.000, 5.000, 6.000, 
                                 7.000, 8.000, 10.00, 12.00, 15.00, 17.00, 20.00, 25.00, 
                                 30.00, 35.00, 40.00, 50.00],
                        description="umin")
alpha = widgets.FloatSlider(value=SED_PARAMETERS['dl2014']['alpha'],
                            min=1., max=3., step=.5, 
                            description="alpha", continuous_update=False)
gamma = widgets.FloatSlider(value=SED_PARAMETERS['dl2014']['gamma'],
                            min=0., max=20., step=1, 
                            description="gamma", continuous_update=False)

redshift = widgets.FloatSlider(value=SED_PARAMETERS['redshifting']['redshift'],
                               min=0, max=10, step=.1, 
                               description="redshift", continuous_update=False)

l_range = widgets.FloatRangeSlider(value=(1.5, 6), min=1, max=8, step=.1, continuous_update=False)

sliders = widgets.VBox([
    widgets.Text("Fors2 galaxy name"),
    name,
    widgets.Text("Log of adjust factor"),
    log_adjust_factor,
    widgets.Text("Star Formation History"),
    tau_main, age_main, tau_burst, age_burst, f_burst,
    widgets.Text("BC03 SSP"),
    metallicity, 
    widgets.Text("Nebular"),
    logU, f_esc, f_dust, 
    widgets.Text("Dust Attenuation"),
    E_BV_lines, E_BV_factor, uv_bump_amplitude, uv_bump_width, powerlaw_slope, Rv, 
    widgets.Text("IR emission"),
    qpah, umin, alpha, gamma, 
    widgets.Text("Redshift"),
    redshift, 
    widgets.Text("Lambda range (log)"),
    l_range
])



figure = widgets.interactive_output(
    plot_sed,
    {
        'name': name,
        'log_adjust_factor': log_adjust_factor,
        'tau_main': tau_main,
        'age_main': age_main,
        'tau_burst': tau_burst,
        'age_burst': age_burst,
        'f_burst': f_burst,
        'metallicity': metallicity,
        'logU': logU,
        'f_esc': f_esc,
        'f_dust': f_dust,
        'E_BV_lines': E_BV_lines,
        'E_BV_factor': E_BV_factor,
        'uv_bump_amplitude': uv_bump_amplitude,
        'uv_bump_width': uv_bump_width,
        'powerlaw_slope': powerlaw_slope,
        'Rv': Rv,
        'qpah': qpah,
        'umin': umin,
        'alpha': alpha,
        'gamma': gamma,
        'redshift': redshift,
        'l_range': l_range,
    }
)

widgets.HBox([sliders, figure])